In [1]:
# vllm
!pip uninstall -y torch
!pip install -q --no-index --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-vllm vllm
!pip install -q -U /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q -U /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q --no-deps --no-index /kaggle/input/hf-libraries/sentence-transformers/sentence_transformers-3.1.0-py3-none-any.whl
#!pip install --no-deps --no-index /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl

# Peft
!pip install transformers peft accelerate \
    -q -U --no-index --find-links /kaggle/input/lmsys-wheel-files
!pip install --no-index /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --find-links=/kaggle/input/bitsandbytes0-42-0

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Looking in links: /kaggle/input/bitsandbytes0-42-0
Processing /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl
Looking in links: /kaggle/input/bitsandbytes0-42-0
Processing /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl
Processing /kaggle/input/bitsandbytes0-42-0/coloredlogs-15.0.1-py2.py3-none-any.whl (from optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/transformers-4.42.4-py3-none-any.whl (from transformers[sentencepiece]<4.43.0,>=4.26.0->optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from transformers<4.43.0,>=4.26.0->transformers[sentencepiece]<4.43.0,>=4.26.0->optimum==1.21.2)
Processing /kaggle/input/bitsandbytes0-42-0/humanfriendly-10.0-py2.py3-none-any.whl (from coloredlogs->optimum==1.21.2)
  Attempting uninstall: tokenizers
    F

## LLM Reasoning
Prompt LLM to identify likely misconception that led to wrong answer

In [2]:
import pandas as pd


df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")
explode_df = []
for idx, row in df.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row["CorrectAnswer"]:
            continue
        correct_option = row["CorrectAnswer"]

        explode_df.append({"QuestionId_Answer": f"{row.QuestionId}_{option}",
                           "ConstructName": row.ConstructName,
                           "SubjectName": row.SubjectName,
                           "QuestionText": row.QuestionText,
                           "CorrectAnswer": row[f"Answer{correct_option}Text"],
                           "IncorrectAnswer": row[f"Answer{option}Text"]
                           })

df = pd.DataFrame(explode_df)
        

In [3]:
from torch.utils.data import Dataset

class MathDataset(Dataset):
    def __init__(self, df):
        self.data = df
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        subjectName = self.data.loc[idx, "SubjectName"]
        constructName = self.data.loc[idx, "ConstructName"]
        question = self.data.loc[idx, "QuestionText"]
        correct_answer = self.data.loc[idx, "CorrectAnswer"]
        wrong_answer = self.data.loc[idx, "IncorrectAnswer"]
        

        prompt = f"""
        Here is a question about {constructName} ({subjectName}):
        
        - Question: {question}
        - Correct Answer: {correct_answer}
        - Wrong Answer: {wrong_answer}
        
        Please identify the likely misconception or reasoning error that led the student to choose the wrong answer. Focus only on explaining the misconception.
        """
    
        message = [
            {"role": "system", "content": "You are a proficient Mathematics teacher. Your goal is to identify the likely misconception or reasoning error that led the student to choose the wrong answer."},
            {"role": "user", "content": prompt.strip()}
        ]
        
        return message, correct_answer, str(self.data.loc[idx, "QuestionId_Answer"])
    
def collate_batch(batch):
    data, labels, question_ids = zip(*batch)
    text = tokenizer.apply_chat_template(
        data,
        tokenize=False,
        add_generation_prompt=True
    )
    return text, labels, question_ids



In [4]:
# import re
# from tqdm import tqdm
# from torch.utils.data import DataLoader
# from vllm import LLM, SamplingParams
# import json

# model_name = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"
# device = "cuda" # the device to load the model onto
# model = LLM(model_name,
#             quantization="awq",
#             gpu_memory_utilization=1,
#             tensor_parallel_size=2,
#             trust_remote_code=True,
#             dtype="half",
#             max_model_len=4000)

# tokenizer = model.get_tokenizer()
# train_dataset = MathDataset(df)
# train_pbar = tqdm(DataLoader(train_dataset, batch_size=32, collate_fn=collate_batch))

# llm_response = []
# for model_inputs, answers, question_ids in train_pbar:
#     outputs = model.generate(
#         model_inputs,
#         SamplingParams(
#             n=1,
#             temperature=0,
#             seed=123,
#             max_tokens=1024
#         ),
#         use_tqdm=False
#     )
    
#     for i in range(len(outputs)):
#         output = outputs[i]
#         llm_response.append({"QuestionId_Answer": question_ids[i], "Misconception": output.outputs[0].text})

# llm_response

In [5]:
import re
from tqdm import tqdm
from torch.utils.data import DataLoader
from vllm import LLM, SamplingParams
import json

model_name = "/kaggle/input/Qwen/Qwen2.5-Math-7B-Instruct"
device = "cuda" # the device to load the model onto
model = LLM(model_name,
            gpu_memory_utilization=1,
            tensor_parallel_size=2,
            trust_remote_code=True,
            dtype="half",
            max_model_len=4000)

tokenizer = model.get_tokenizer()
train_dataset = MathDataset(df)
train_pbar = tqdm(DataLoader(train_dataset, batch_size=32, collate_fn=collate_batch))

llm_response = []
for model_inputs, answers, question_ids in train_pbar:
    outputs = model.generate(
        model_inputs,
        SamplingParams(
            n=1,
            temperature=0,
            seed=123,
            max_tokens=2048
        ),
        use_tqdm=False
    )
    
    for i in range(len(outputs)):
        output = outputs[i]
        llm_response.append({"QuestionId_Answer": question_ids[i], "Misconception": output.outputs[0].text})

llm_response

2024-11-23 02:30:10,964	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


OSError: Incorrect path_or_model_id: '/kaggle/input/Qwen/Qwen2.5-Math-7B-Instruct'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
import torch
import gc


del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()
!nvidia-smi

## Compute cosine similarity of embedding
Compute embeddings of **MisconceptionName** and LLM reasoned possible misconceptions using [SFR-Embedding-Mistral](https://huggingface.co/Salesforce/SFR-Embedding-Mistral)

In [ ]:
from torch import Tensor
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
import torch
from tqdm import trange
import numpy as np


def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch


def last_token_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])


def get_detailed_instruct(task_description: str, query: str) -> str:
    return f"Instruct: {task_description}\nQuery: {query}"


def inference(df, misconception_df, device):
    task = "Given the likely misconception or reasoning error that led the student to choose the wrong answer, please retrieve the most accurate description of the misconception."
    quries = [get_detailed_instruct(task, str(q)) for q in df["Misconception"].values]
    passages = [str(mis) for mis in misconception_df["MisconceptionName"].values]
   
    # load model and tokenizer
    lora_path="/kaggle/input/v7-recall/epoch_19_model/adapter.bin"
    model_path = "/kaggle/input/sfr-embedding-mistral/SFR-Embedding-2_R"
    tokenizer = AutoTokenizer.from_pretrained(lora_path.replace("/adapter.bin", ""))
    bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16                  
            )
    
    backbone = AutoModel.from_pretrained(model_path, quantization_config=bnb_config, device_map=device)
    config = LoraConfig(
            r=64,
            lora_alpha=128,
            target_modules=[
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "gate_proj",
                "up_proj",
                "down_proj",
            ],
            bias="none",
            lora_dropout=0.05,  # Conventional
            task_type="CAUSAL_LM",
        )
    model = get_peft_model(backbone, config)
    d = torch.load(lora_path, map_location=model.device)
    model.load_state_dict(d, strict=False)
    model = model.eval()
    model = model.to(device)

    max_length = 4096
    input_texts = quries + passages
    batch_size = 32
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(text) for text in input_texts])
    input_texts_sorted = [input_texts[idx] for idx in length_sorted_idx]
    
    for start_index in trange(0, len(input_texts), batch_size, desc="Batches", disable=False):
        batch_dict = tokenizer(input_texts_sorted[start_index: start_index + batch_size], max_length=max_length, padding=True, truncation=True, return_tensors="pt")
        batch_dict = batch_to_device(batch_dict, device)  
        with torch.no_grad():
            outputs = model.model(**batch_dict)
            
        embeddings = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        embeddings = embeddings.detach().cpu()
        embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
        all_embeddings.extend(embeddings.detach().cpu())

    all_embeddings = np.array([all_embeddings[idx] for idx in np.argsort(length_sorted_idx)])
    print(all_embeddings.shape)
    num_quries = len(quries)
    scores = (all_embeddings[:num_quries] @ all_embeddings[num_quries:].T) * 100
    print(scores.shape)
    top_25 = np.argsort(-scores, axis=1)[:, :25] # top 25 similar misconception
    return top_25 

In [ ]:
from peft import (
    LoraConfig,
    get_peft_model,
)

llm_misconception = pd.DataFrame(llm_response)
df["Misconception"] = llm_misconception["Misconception"]
misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")
device = "cuda"

top_25 = inference(df, misconception_df, device)
top_25

In [ ]:
submission = []
for idx, row in df.iterrows():
    submission.append({"QuestionId_Answer": row["QuestionId_Answer"], "MisconceptionId": " ".join(top_25[idx].astype(str))})
pd.DataFrame(submission).to_csv("submission.csv", index=False)

